In [2]:
import os
from openai import OpenAI
import openai
from PIL import Image
import base64

In [3]:
os.environ["OPENAI_API_KEY"] = "sk-6OyTxsQcuf5tBz4w221b5822EfA143539f03A7F7BaB778D4"
os.environ["OPENAI_BASE_URL"] = "https://api.xiaoai.plus/v1"

client = OpenAI()

GPT_MODEL = "gpt-4o"

prompt = "Generates prompt that is inspirational and suggestive, it can be used to recreate an uploaded image exactly"
# prompt = "Carefully analyze the provided image and generate an exhaustive description that captures every element essential for its recreation. Detail the spatial arrangement, key subjects, and any background elements. Specify the color palette, noting any gradients or unique shades present. Describe any actions or interactions between subjects, and capture the mood or atmosphere effectively. Your description should be precise and comprehensive, enabling an artist or a generative model to replicate the image with high fidelity based on your text alone."

In [4]:
# Encode the image at the given path to a base64 string
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Get the base64 encoded description of an image at the given path
def get_image_description(image_path):
    encoded_image = encode_image(image_path)
    return encoded_image

# Generate a detailed description of the image using GPT-4
def generate_description_with_gpt(base64_image, prompt):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    },
                ]
            }
        ],
        stream=False,
        max_tokens=300,
    )
    summary = response.choices[0].message.content.strip()
    return summary 

# Process a list of image paths and generate descriptions for each
def process_images(image_paths):
    descriptions = []
    for image_path in image_paths:
        image_description = get_image_description(image_path)
        description = generate_description_with_gpt(image_description, prompt)
        descriptions.append(description)
    return descriptions

In [ ]:
def get_images_path(set_path):

    file_paths = []

    for root, _, files in os.walk(set_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    return file_paths

In [5]:
if __name__ == "__main__":
    set_path = "./images"
    image_paths = get_images_path(set_path)
    descriptions = process_images(image_paths)
    for i, description in enumerate(descriptions):
        print(f"Description for image {i+1}: {description}")

Description for image 1: The image captures a scene featuring two lions on a grassy savanna. Here is a detailed description:

### Key Subjects:
1. **Male Lion:**
   - **Position and Action:** The male lion is mounted on the female lion from behind, suggestive of mating behavior.
   - **Appearance:** The male lion has a large, full mane that is a rich golden-brown color, with highlights of darker brown. His facial expression is open-mouthed, bearing teeth, and resembling a snarl or roar. His eyes are partially closed, emphasizing the ferocity or intensity of his expression.

2. **Female Lion:**
   - **Position and Action:** The female lion is lying on the ground on her stomach. She appears calm, with her front paws stretched out in front of her, her head upright, and her eyes open wide, staring directly into the camera.
   - **Appearance:** The female lion's coat is a light golden-tan color. Her expression is somewhat neutral or resigned, contrasting with the male's aggressive demeanor.